In [22]:
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
from dash import Dash, html, dcc, Input, Output, no_update, callback, State, callback_context
from skimage import data, draw
from scipy import ndimage
from skimage import io
import os
import json

from geemap import ee_to_numpy

In [12]:
from google.oauth2 import service_account
import ee
from pathlib import Path

# Define the required scopes
scopes = ['https://www.googleapis.com/auth/earthengine', 
          'https://www.googleapis.com/auth/devstorage.read_write']

# Use the service account JSON key path and scopes 
credentials = service_account.Credentials.from_service_account_file(
    # '/home/users/leamhowe/ee-credentials/ee-snow-patches-81eaf6e60bf9.json',
    Path(r'C:\Users\s2112771\Documents\ee-credentials\ee-snow-patches-81eaf6e60bf9.json'),
    scopes=scopes)

# Initialize the Earth Engine API with the credentials
ee.Initialize(credentials)
ee.Authenticate()

True

In [30]:
def get_false_color_image(timestep_idx):
    """Helper function to get false color image for a given timestep index"""
    image = ee.Image(image_list.get(timestep_idx))
    # image = set_clouds_to_value(image)  # Apply cloud mask
    bands = ['B11', 'B8', 'B3'] # B11 (SWIR), B4 (Red), B3 (Green)... good for distinguishing snow from cloud
    # bands = ['B4', 'B3', 'B2'] # B11 (SWIR), B4 (Red), B3 (Green)... good for distinguishing snow from cloud
    resampled = image.select(bands).resample().reproject(image.select('B2').projection())
    region_image = resampled.clip(aoi)
    image_array = ee_to_numpy(region_image, region=aoi, scale=pixel_size) 

    image_id = image.get('system:index').getInfo()  # Get the image ID

    return image_array, image_id

def get_true_color_image(timestep_idx):
    """Helper function to get false color image for a given timestep index"""
    image = ee.Image(image_list.get(timestep_idx))
    # image = set_clouds_to_value(image)  # Apply cloud mask
    # bands = ['B11', 'B4', 'B3'] # B11 (SWIR), B4 (Red), B3 (Green)... good for distinguishing snow from cloud
    bands = ['B4', 'B3', 'B2'] # true color bands
    resampled = image.select(bands).resample().reproject(image.select('B2').projection())
    region_image = resampled.clip(aoi)
    image_array = ee_to_numpy(region_image, region=aoi, scale=pixel_size) 

    image_id = image.get('system:index').getInfo()  # Get the image ID

    return image_array, image_id

In [19]:
location, lon, lat = 'Garbh_Choire_Mor',  -3.744673,  57.060077

# -36.39952	148.31508	2030	2	Blue Lake Upper	NSW
# -36.40363	148.31204	1905	4	Blue Lake Lower	NSW
# location, lon, lat = 'Blue Lake Upper', 148.31508, -36.39952
# location, lon, lat = 'Blue Lake Lower', 148.31204, -36.40363

start_date = '2025-07-12'
end_date = '2025-07-14'
original_size=100

# Satellite Data
# da = cubo.create(lat=lat, lon=lon, collection=ee.ImageCollection("COPERNICUS/S2_HARMONIZED"),
#                     bands=s2l1c_bands_gee, 
#                     start_date=start_date, end_date=end_date,
#                     edge_size=original_size, resolution=10, gee=True, chunk_size=1000,)

# using gee python API
# create a point geometry for the location
point = ee.Geometry.Point([ lon, lat]) 
# Define pixel size and region dimensions
pixel_size = 10  # Sentinel-2 native resolution (meters)
region_size = 98  # 100 pixels
buffer_distance = (region_size * pixel_size) / 2  # 500 meters (100px * 10m/px / 2)

# Create a square region around the point
aoi = point.buffer(buffer_distance).bounds()

In [20]:

s2_toa = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') # Sentinel-2 TOA

filtered = (s2_toa
    .filterBounds(point).filterDate(start_date, end_date)
    # .linkCollection(csPlus, ['cs_cdf'])  # Join with Cloud Score+
    # .map(set_clouds_to_value)  # Apply cloud replacement
    # .first()  # Get the first image
)
# get a list of the images in the collection
image_list = filtered.toList(filtered.size())
# Get the number of images in the collection
count = image_list.size().getInfo()

In [ ]:
fc_image, image_id = get_false_color_image(0)
rgb_image, image_id = get_true_color_image(0)
vmin = np.percentile(rgb_image, 1)
vmax = np.percentile(rgb_image, 99)
fig = px.imshow(rgb_image, binary_string=True, origin='lower', 
                zmin=vmin, zmax=vmax, aspect=3.5/5.5,
                title=image_id 
                )


fig.update_xaxes(showticklabels=False, title=None)
fig.update_yaxes(showticklabels=False, title=None)

fig.show()

In [37]:
vmin_fc

np.float64(731.0)

In [40]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Get images
fc_image, image_id = get_false_color_image(0)
rgb_image, _ = get_true_color_image(0)

# Normalize RGB image manually
vmin_rgb = np.percentile(rgb_image, 1)
vmax_rgb = np.percentile(rgb_image, 99)
rgb_image_scaled = np.clip((rgb_image - vmin_rgb) / (vmax_rgb - vmin_rgb), 0, 1)
rgb_image_scaled = (rgb_image_scaled * 255).astype(np.uint8)

# Normalize FC image manually
vmin_fc = np.percentile(fc_image, 1)
vmax_fc = np.percentile(fc_image, 99)
fc_image_scaled = np.clip((fc_image - vmin_fc) / (vmax_fc - vmin_fc), 0, 1)
fc_image_scaled = (fc_image_scaled * 255).astype(np.uint8)

# Create subplot
fig = make_subplots(rows=1, cols=2, subplot_titles=("True Color", "False Color"))

# Add True Color
fig.add_trace(go.Image(z=rgb_image_scaled), row=1, col=1)

# Add False Color
fig.add_trace(go.Image(z=fc_image_scaled), row=1, col=2)

# Hide axes, update layout
fig.update_layout(
    title_text=f"Image ID: {image_id}",
    showlegend=False,
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.update_xaxes(showticklabels=False, title=None)
fig.update_yaxes(showticklabels=False, title=None)

fig.show()


In [57]:
def get_image_array(timestep_idx, bands, region, scale, image_list):
    """Fetch an image and return it as a NumPy array."""
    image = ee.Image(image_list.get(timestep_idx))
    resampled = image.select(bands).resample().reproject(image.select('B2').projection())
    region_image = resampled.clip(region)
    image_array = ee_to_numpy(region_image, region=region, scale=scale)
    image_id = image.get('system:index').getInfo()
    return image_array, image_id


In [69]:
# For true color:
rgb_image, image_id = get_image_array(0, bands=['B4', 'B3', 'B2'], region=aoi, scale=pixel_size, image_list=image_list)

# For false color:
fc_image, _ = get_image_array(0, bands=['B11', 'B4', 'B3'], region=aoi, scale=pixel_size, image_list=image_list)


In [71]:
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np

# Get images
rgb_image, image_id = get_image_array(0, ['B4', 'B3', 'B2'], aoi, pixel_size, image_list)
fc_image, _ = get_image_array(0, ['B11', 'B4', 'B3'], aoi, pixel_size, image_list)

# Compute contrast bounds
vmin_rgb = np.percentile(rgb_image, 1)
vmax_rgb = np.percentile(rgb_image, 99)
vmin_fc = np.percentile(fc_image, 1)
vmax_fc = np.percentile(fc_image, 99)

# Use px.imshow for contrast stretching, then extract trace
fig_rgb = px.imshow(rgb_image, zmin=vmin_rgb, zmax=vmax_rgb, binary_string=True)
fig_fc = px.imshow(fc_image, zmin=vmin_fc, zmax=vmax_fc, binary_string=True)

# Create subplot
fig = make_subplots(rows=1, cols=2, subplot_titles=["True Color", "False Color"])
fig.add_trace(fig_rgb.data[0], row=1, col=1)
fig.add_trace(fig_fc.data[0], row=1, col=2)

# Layout adjustments
fig.update_layout(
    title_text=f"Image ID: {image_id}",
    title={'x': 0.5},
    height=600,
    margin=dict(l=0, r=0, t=80, b=10)
)

# Preserve aspect ratio
fig.update_yaxes(scaleanchor="x", row=1, col=1)
fig.update_yaxes(scaleanchor="x", row=1, col=1)

# Remove axis ticks
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

fig.show()


In [72]:
import geemap

# Plot the true color image using geemap
geemap.imshow(rgb_image, title=f"True Color - {image_id}")

AttributeError: module 'geemap' has no attribute 'imshow'

In [75]:
import geemap
import ee

def plot_image_geemap(timestep_idx, bands, vis_params=None, region=None, zoom=10):
    """Display a single image from image_list on an interactive geemap map."""
    image = ee.Image(image_list.get(timestep_idx))
    image = image.select(bands)

    # Center on the region if given, else center on image centroid
    # if region:
    #     center = region.centroid().coordinates().getInfo()
    # else:
    #     center = image.geometry().centroid().coordinates().getInfo()
    center = region.centroid().coordinates().getInfo() if region else image.geometry().centroid().coordinates().getInfo()

    # Set default visualization parameters if none provided
    if vis_params is None:
        vis_params = {
            'min': 0,
            'max': 3000,  # Sentinel-2 reflectance scale
            'bands': bands
        }

    # Create the map
    Map = geemap.Map(center=center[::-1], zoom=zoom)  # reverse lat/lon for center
    Map.addLayer(image.clip(region if region else image.geometry()), vis_params, 'Image')
    Map.addLayer(region, {}, 'AOI') if region else None
    Map.addLayerControl()
    return Map

# Example usage: True color
true_color_map = plot_image_geemap(
    timestep_idx=0,
    bands=['B4', 'B3', 'B2'],  # true color
    vis_params={'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    region=aoi,
    zoom=10
)
true_color_map


EEException: Geometry.centroid: Unable to perform this geometry operation. Please specify a non-zero error margin.